<a href="https://colab.research.google.com/github/youjin0512/study_data_analytics/blob/main/docs/quests/CDAs/CDA_fruitstore_saleshistory_mixed_Normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 한글 폰트 설치

In [41]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab 경우 Reuntime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

## matplotlib 시각화

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

## load dataset

In [31]:
import pandas as pd
df_sales = pd.read_csv('fruitstore_saleshistory.csv')
df_sales[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1.0,1.0,2.723699e+08,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207.0,매장C,NaN,10:04
1,2018-01-02,화,1.0,1.0,1.506656e+09,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977.0,매장C,NaN,10:05


## 범주형 / 연속형 : 범주형에 따른 관계 확인

### 연속형이 정규 분포이고, 등분산 일 때

### 예제 가설 : 물품중분류, 구매금액 (정규분포)
- 분석 내용 : 각 과일 종류(범주형)별 평균 판매 금액(연속형)에 차이가 있는가?
 + 귀무 가설 : 각 과일 종류별 평균 판매 금액에 차이가 없다.
 + 대립 가설 : 각 과일 종류별 평균 판매 금액에 차이가 있다.

### 등분산 검증(levene)

In [36]:
df_sales_fruit = df_sales[df_sales['물품중분류']=="과일"]
fruit_list = df_sales_fruit['물품소분류'].unique()
fruit_list

array(['사과', '건대추', '귤(조생)', '배', '산딸기', '단감', '참다래', '레몬', '블루베리', '곶감',
       '귤만감', '냉동건조과일', '홍시용감(대봉)'], dtype=object)

In [37]:
df_sales_mean_fruit = df_sales_fruit.groupby('물품소분류')['구매금액'] # 과일 종류별 평균 판매금액
df_sales_mean_fruit

#### ANOVA TEST(f_oneway) : 범주형 3집단 이상
- 연속형이 '정규분포'이면서 범주형에 따른 항목이 '등분산', 그리고 항목 개수가 3개 이상

In [38]:
series_list = list()  # list 초기화
for fruit in fruit_list:
  condition = f'물품소분류 == "{fruit}"'   # 앞의 물품소분류 : 데이터 프레임 column의 채널    # f{fruit} : 변수로 인지
  series_amount = df_sales_fruit.query(condition)['구매금액'].dropna()   # condition에 따른 '구매금액' series 작성  # dropna : 결측치 제거
  series_list.append(series_amount)
series_list[1].iloc[:5]   # series_list의 인덱스1, 즉 2번째 항목(홈쇼핑/방송)에 대한 5개까지 출력

62      22973.0
3066    22973.0
3866    22973.0
4515    22973.0
4763    22973.0
Name: 구매금액, dtype: float64

In [39]:
import scipy.stats as stats
stats.f_oneway(*series_list)   # series 일떄 * 1개 부여하면, list 전체 넘어감

F_onewayResult(statistic=6.65988663849486, pvalue=2.640715984250167e-11)

e는 지수를 나타내는 기호로, '10의 거듭제곱'을 의미합니다. 따라서 e-11은 '10의 -11제곱'을 나타냅니다.

2.640715984250167e-11은 2.640715984250167 * 10^-11로 해석됩니다. 즉, 2.640715984250167를 10의 -11제곱만큼 곱한 값입니다.

ANOVA는 귀무가설(H0)과 대립가설(H1)을 설정하여 가설검정을 수행합니다.

+ 귀무가설: 모든 그룹의 평균은 동일하다.
+ 대립가설: 적어도 한 그룹의 평균은 다른 그룹과 다르다.

ANOVA는 F-통계량을 계산하여 이 가설들을 검정합니다. F-통계량은 그룹 간 분산(그룹의 평균들 사이의 차이를 측정)을 그룹 내 분산(각 그룹 내의 개별 관측값들이 그룹의 평균으로부터 얼마나 떨어져 있는지 측정)으로 나눈 값입니다. 그룹 간 분산이 그룹 내 분산보다 크면 F-통계량은 1보다 크고, 그룹의 평균들 사이에 차이가 있다는 증거가 됩니다.

p-value는 귀무가설 하에서 현재 관측된 통계치와 같거나 더 극단적인 통계치가 관찰될 확률을 나타냅니다. p-value가 특정 임계값(일반적으로 0.05)보다 작으면 귀무가설을 기각하고 대립가설을 수용합니다. 즉, 적어도 한 그룹의 평균은 다른 그룹과 다르다는 결론을 내립니다.

여기서 주어진 F-통계량 값은 약 6.66이고, p-value는 약 2.64e-11입니다. 이 p-value는 매우 작은 값으로, 0.05보다 훨씬 작습니다. 따라서 이 경우에는 귀무가설을 기각하고, 적어도 한 종류의 과일의 평균 판매 금액이 다른 종류의 과일과 다르다는 결론을 내릴 수 있습니다.

In [40]:
round(2.640715984250167e-11, 6)

0.0

round() 함수는 파이썬에서 제공하는 내장 함수로, 특정 숫자를 반올림하는데 사용됩니다. round() 함수는 두 개의 매개변수를 받을 수 있습니다.

첫 번째 매개변수는 반올림할 숫자입니다.
두 번째 매개변수는 선택적이며, 반올림할 소수점 이하의 자릿수를 지정합니다. 이 매개변수가 생략되면, 입력된 숫자는 가장 가까운 정수로 반올림됩니다.
예를 들어, round(2.640715984250167e-11, 6)은 숫자 2.640715984250167e-11를 소수점 아래 6자리까지 반올림한 결과를 반환합니다. 따라서 이 함수의 결과는 0.000000이 될 것입니다.

이는 2.640715984250167e-11이 매우 작은 수이기 때문에 소수점 아래 6자리에서 반올림하면 0.000000이 나오기 때문입니다.

ANOVA 테스트의 결과는 F 통계량과 p-value 두 가지를 제공합니다.

F 통계량(statistic=6.65988663849486): 이것은 그룹 간의 분산(평균 간 차이)이 그룹 내 분산(각 그룹 내에서의 개별 관측치와 그룹 평균 간의 차이)에 비해 얼마나 큰지를 측정하는 값입니다. F 통계량이 크면 클수록 그룹 간의 평균 차이가 크다는 것을 나타냅니다. 여기서 F 통계량이 6.66이므로, 이는 그룹 간의 평균 차이가 그룹 내에서의 변동성에 비해 상대적으로 크다는 것을 나타냅니다.
p-value(pvalue=2.640715984250167e-11): p-value는 귀무가설(모든 그룹의 평균이 동일하다) 하에서 현재 관측된 통계치와 같거나 더 극단적인 통계치가 관찰될 확률을 나타냅니다. 일반적으로 p-value가 0.05 미만이면 귀무가설을 기각하고, 대립가설(적어도 한 그룹의 평균은 다르다)을 수용합니다. 여기서 p-value는 2.64e-11로, 이는 0.05보다 훨씬 작은 값입니다. 따라서 귀무가설을 기각하고, 대립가설을 수용합니다.
따라서 이 ANOVA 테스트 결과를 통해 '각 과일 종류별 평균 판매 금액에는 통계적으로 유의미한 차이가 있다'는 결론을 내릴 수 있습니다. 즉, 과일 종류에 따라 평균 판매 금액이 다르다는 가설이 데이터에 부합한다는 것을 알 수 있습니다.

# 분석 결과
 - 통계 결론 : 귀무 가설 기각, 대립 가설 성립
 - 사실 결론 : 정확한 수치는 확실하지 않지만, 각 과일 종류별 평균 판매 금액에 유의미한 차이가 있다.